# ResNet For CIFAR0-10

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/DL_Mini_Proj
!ls

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/DL_Mini_Proj'
/content
drive  sample_data


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
#import renet

import torchvision
import torchvision.transforms as transforms

import os
from torchsummary import summary
import torch.utils.data as data

### Model Construction

In [84]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes) # ochange

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [85]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ResNet18().to(device)
model.layer4 = torch.nn.Sequential(*[model.layer4[0]]) # https://discuss.pytorch.org/t/how-to-delete-layer-in-pretrained-model/17648/4
model.layer3 = torch.nn.Sequential(*[model.layer3[0]])
model.layer1 = torch.nn.Sequential(*[model.layer1[0]])
model.layer2 = torch.nn.Sequential(*[model.layer2[0]])

In [86]:
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8          [-1, 128, 16, 16]          73,728
       BatchNorm2d-9          [-1, 128, 16, 16]             256
           Conv2d-10          [-1, 128, 16, 16]         147,456
      BatchNorm2d-11          [-1, 128, 16, 16]             256
           Conv2d-12          [-1, 128, 16, 16]           8,192
      BatchNorm2d-13          [-1, 128, 16, 16]             256
       BasicBlock-14          [-1, 128,

Construction Complete

### Data Preprocessing

In [87]:
class Cutout(object):
  """Randomly mask out one or more patches from an image.
  Args:
      n_holes (int): Number of patches to cut out of each image.
      length (int): The length (in pixels) of each square patch.
  """
  def __init__(self, n_holes, length):
      self.n_holes = n_holes
      self.length = length

  def __call__(self, img):
      """
      Args:
          img (Tensor): Tensor image of size (C, H, W).
      Returns:
          Tensor: Image with n_holes of dimension length x length cut out of it.
      """
      h = img.size(1)
      w = img.size(2)

      mask = np.ones((h, w), np.float32)

      for n in range(self.n_holes):
          y = np.random.randint(h)
          x = np.random.randint(w)

          y1 = np.clip(y - self.length // 2, 0, h)
          y2 = np.clip(y + self.length // 2, 0, h)
          x1 = np.clip(x - self.length // 2, 0, w)
          x2 = np.clip(x + self.length // 2, 0, w)

          mask[y1: y2, x1: x2] = 0.

      mask = torch.from_numpy(mask)
      mask = mask.expand_as(img)
      img = img * mask

      return img

In [89]:
def load_CIFAR10(batch_size, train_ratio):

  ROOT = './data'
  trainset = torchvision.datasets.CIFAR10(
      root = ROOT,
      train = True, 
      download = True
  )

  # Compute means and standard deviations
  means = trainset.data.mean(axis=(0,1,2)) / 255
  stds = trainset.data.std(axis=(0,1,2)) / 255
  #print(means, stds)

  # Preprocess setting
  transform_train = transforms.Compose([
      transforms.RandomCrop(32, padding=4),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean=means, std=stds),
      Cutout(n_holes=1, length=16)
  ])
  transform_test = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=means, std=stds)
  ])

  # Load the dataset
  trainset = torchvision.datasets.CIFAR10(
      root = ROOT, 
      train = True, 
      download = True, 
      transform = transform_train
  )
  testset = torchvision.datasets.CIFAR10(
      root = ROOT, 
      train = False, 
      download = True, 
      transform = transform_test
  )

  train_iterator = data.DataLoader(trainset, batch_size)
  test_iterator = data.DataLoader(testset, batch_size)

  return train_iterator, test_iterator
  """
  # Split trainset for validset
  n_train = int(len(trainset) * train_ratio)
  n_valid = len(trainset) - n_train
  train_dataset, valid_dataset = data.random_split(trainset, [n_train, n_valid])
  
  # Build dataloader
  train_iterator = data.DataLoader(train_dataset, batch_size)
  valid_iterator = data.DataLoader(valid_dataset, batch_size)
  test_iterator = data.DataLoader(testset, batch_size)

  return train_iterator, valid_iterator, test_iterator
  """

In [90]:
# trainloader, validloader, testloader = load_CIFAR10(batch_size=16, train_ratio=1)
trainloader, testloader = load_CIFAR10(batch_size=16, train_ratio=1)

100%|██████████| 170498071/170498071 [00:01<00:00, 102247498.28it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [91]:
"""
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
"""

'\ntransform_train = transforms.Compose([\n    transforms.RandomCrop(32, padding=4),\n    transforms.RandomHorizontalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n\ntransform_test = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n'

In [92]:
"""
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train) # change transform in future
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test) # change transform in future
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
"""

"\ntrainset = torchvision.datasets.CIFAR10(\n    root='./data', train=True, download=True, transform=transform_train) # change transform in future\ntrainloader = torch.utils.data.DataLoader(\n    trainset, batch_size=128, shuffle=True, num_workers=2)\n\ntestset = torchvision.datasets.CIFAR10(\n    root='./data', train=False, download=True, transform=transform_test) # change transform in future\ntestloader = torch.utils.data.DataLoader(\n    testset, batch_size=100, shuffle=False, num_workers=2)\n\nclasses = ('plane', 'car', 'bird', 'cat', 'deer',\n           'dog', 'frog', 'horse', 'ship', 'truck')\n"

In [94]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)#,weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [95]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print("Train Set Loss:",train_loss/total)



In [96]:
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc
    print('Test Set Accuracy:',acc)
    return acc


In [97]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
Train Set Loss: 0.11250551059007645
Saving..
Test Set Accuracy: 45.42

Epoch: 1
Train Set Loss: 0.08646546324849129
Saving..
Test Set Accuracy: 56.74

Epoch: 2
Train Set Loss: 0.07316700073242187
Saving..
Test Set Accuracy: 65.77

Epoch: 3
Train Set Loss: 0.06538874449670315
Saving..
Test Set Accuracy: 70.77

Epoch: 4
Train Set Loss: 0.059088009424209595
Saving..
Test Set Accuracy: 73.96

Epoch: 5
Train Set Loss: 0.053653905138671396
Saving..
Test Set Accuracy: 75.2

Epoch: 6
Train Set Loss: 0.04969362676978111
Saving..
Test Set Accuracy: 77.02

Epoch: 7
Train Set Loss: 0.04623111477643251
Saving..
Test Set Accuracy: 80.83

Epoch: 8
Train Set Loss: 0.043636488420963286
Saving..
Test Set Accuracy: 81.01

Epoch: 9
Train Set Loss: 0.04136169065803289
Saving..
Test Set Accuracy: 81.65

Epoch: 10
Train Set Loss: 0.039682828458100554
Saving..
Test Set Accuracy: 82.05

Epoch: 11
Train Set Loss: 0.037984485948830846
Saving..
Test Set Accuracy: 83.23

Epoch: 12
Train Set Loss: 0.03655

In [ ]:
"""
for g in optimizer.param_groups:
    print(optimizer.param_groups[0]['lr'])
    g['lr'] = 0.001
"""
state = {
      'net': model.state_dict()
}
if not os.path.isdir('checkpoint'):
  os.mkdir('checkpoint')
torch.save(state, './checkpoint/20Epoch.pth')

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-6)

In [ ]:
start_epoch = 20  

for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 20
Train Set Loss: 0.06111984935224056
Test Set Accuracy: 76.53

Epoch: 21
Train Set Loss: 0.056872786796987054
Test Set Accuracy: 77.37

Epoch: 22
Train Set Loss: 0.054467707093656066
Test Set Accuracy: 78.47

Epoch: 23
Train Set Loss: 0.052736565395891666
Test Set Accuracy: 79.78

Epoch: 24
Train Set Loss: 0.05089716591179371
Saving..
Test Set Accuracy: 79.98

Epoch: 25
Train Set Loss: 0.04973517975986004
Saving..
Test Set Accuracy: 80.31

Epoch: 26
Train Set Loss: 0.048367037560641764
Saving..
Test Set Accuracy: 80.35

Epoch: 27
Train Set Loss: 0.047496571877896784
Saving..
Test Set Accuracy: 81.09

Epoch: 28
Train Set Loss: 0.04673378049641848
Saving..
Test Set Accuracy: 81.87

Epoch: 29
Train Set Loss: 0.04588644198894501
Test Set Accuracy: 81.37

Epoch: 30
Train Set Loss: 0.04482184986442327
Test Set Accuracy: 81.79

Epoch: 31
Train Set Loss: 0.04400566786378622
Saving..
Test Set Accuracy: 83.04

Epoch: 32
Train Set Loss: 0.04339637280225754
Test Set Accuracy: 83.0

Epoch

In [ ]:
start_epoch = 40  

for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 40
Train Set Loss: 0.03933024534359574
Test Set Accuracy: 84.0

Epoch: 41
Train Set Loss: 0.03863885339826346
Test Set Accuracy: 83.97

Epoch: 42
Train Set Loss: 0.038449449110776183
Saving..
Test Set Accuracy: 84.91

Epoch: 43
Train Set Loss: 0.03798370545908809
Test Set Accuracy: 84.66

Epoch: 44
Train Set Loss: 0.037390205317288636
Test Set Accuracy: 84.39

Epoch: 45
Train Set Loss: 0.036924358177632094
Test Set Accuracy: 84.75

Epoch: 46
Train Set Loss: 0.036688906094133854
Test Set Accuracy: 84.78

Epoch: 47
Train Set Loss: 0.03652169727616012
Saving..
Test Set Accuracy: 85.04

Epoch: 48
Train Set Loss: 0.03618277742251754
Saving..
Test Set Accuracy: 85.47

Epoch: 49
Train Set Loss: 0.03565346192106605
Test Set Accuracy: 85.18

Epoch: 50
Train Set Loss: 0.035647349169552325
Test Set Accuracy: 85.31

Epoch: 51
Train Set Loss: 0.03535126780733466
Test Set Accuracy: 84.88

Epoch: 52
Train Set Loss: 0.03477174524277449
Saving..
Test Set Accuracy: 85.71

Epoch: 53
Train Set Los

In [ ]:
print('Accuracy: ',test(0))

Saving..
Accuracy: 10.0
Accuracy:  10.0
